# Frequent Words with Mismatches

http://rosalind.info/problems/ba1i/

### Plan:
1. get all possible *k*-mers from original sequence
2. generate all possible (up-to *d* changes) *k*-mers for each item at point 1
3. find most frequent *k*-mer among all those generated at point 2
4. bam!

In [1]:
from collections import Counter

In [2]:
def find_frequent_kmers_in(sequence, k, d, bases='ATCG'):
    """Find the most frequent k-mers with mismatches in a sequence.
    
    Given a genome `sequence`, finds all the k-mers of length `k` and
    Hamming distance `d` in the genome and returns the most frequent ones.

    Parameters
    ----------
    sequence : str
        Genome string to be analyzed.
    k : int
        Length of the k-mers.
    d : int
        Maximum Hamming distance.
    bases : 'ATCG', optional
        All possible bases.
        
    Returns
    -------
    most_freq_kmers : list of str
        A list of the most frequent k-mers in `sequence`,
        with up to `d` mismatches.
    
    Examples
    --------
    >>> find_frequent_kmers_in('ACGTTGCATGTCGCATGATGCATGAGAGCT', 4, 1)
    ['ATGT', 'ATGC', 'GATG']
    
    """
    
    # 1. get all possible k-mers of length k from sequence
    kmers = find_kmers(sequence, k)
    
    # 2. find all possible unique -- use list(set(..)) -- mutations 
    # for each k-mer
    mutated_kmers = [list(set(mutations(kmer, d, charset=bases))) for kmer in kmers]
    
    # mutated_kmers is a nested list, ie. [[k-mer, k-mer, ...], [k-mer, k-mer, ...], ...]
    # so we need to flatten it
    flat_mutated_kmers = [item for sublist in mutated_kmers for item in sublist]
    
    # build counter of type {k-mer : occurances, k-mer : occurances, ...}
    frequency_counter = Counter(flat_mutated_kmers)
    
    # get a list of tuples ordered by decreasing frequency, ie. [(kmer, 10), (kmer, 9), ...]
    kmer_freq = frequency_counter.most_common()
    
    # get maximum frequency, ie 10 in our example
    max_freq = kmer_freq[0][1]
    
    # 3. return only the k-mers corresponding to the max frequency
    most_freq_kmers = []
    for (kmer, freq) in kmer_freq:
        if freq != max_freq:
            break # exit the for loop
        # otherwise
        most_freq_kmers.append(kmer)
    return most_freq_kmers

In [3]:
def find_kmers(sequence, k):
    """Finds all k-mers of length `k` in a given character `sequence`.
    
    >>> find_kmers('GTAGAGCTGT', 5)
    ['GTAGA', 'TAGAG', 'AGAGC', 'GAGCT', 'AGCTG', 'GCTGT']
    """
    
    kmers = []
    n = len(sequence)

    # for string of length n, n-k+1 possibilities
    for i in range(n-k+1):
        kmers.append(sequence[i:i+k])

    return kmers

In [4]:
from itertools import combinations, product

In [5]:
def mutations(kmer, d, charset='ATCG'):
    """Generates all mutations of `kmer` within Hamming distance `d`, using alphabet `charset`.
    
    Args:
        kmer (str): Input text string.
        d (int): Maximum Hamming distance.
        charset (str): Available alphabet to chose from.
        
    Yields:
        str: The next mutation of `kmer` in the sequence.

    Example:
    >>> list(mutations('GTAGA', 1))
    ['ATAGA', 'TTAGA', 'CTAGA', 'GTAGA', 'GAAGA', 'GTAGA', 'GCAGA', 'GGAGA', 'GTAGA', 'GTTGA', 'GTCGA', 'GTGGA', 'GTAAA', 'GTATA', 'GTACA', 'GTAGA', 'GTAGA', 'GTAGT', 'GTAGC', 'GTAGG']
    """
    k = len(kmer)

    # combinations() is a list of length binom(k, d)
    # of form [(i_1, i_2, ..., i_d), ...]
    for indices in combinations(range(k), d):

        # product is a list of length len(charset)**d, ie. 4**d
        # of the form [(char_1, char_2, ..., char_d), ...]
        for replacements in product(charset, repeat=d):

            mutation = list(kmer)  # convert string to list

            # d elements in zip(): [(i_1, char_1), ..., (i_d, char_d)]
            for index, replacement in zip(indices, replacements): 
                mutation[index] = replacement

            yield "".join(mutation) # binom(k, d) x len(charset)**d non-unique results

# Testing

In [6]:
# for testing on multiple datasets
import unittest 

In [7]:
class TestMismatch(unittest.TestCase):
    def test_1(self):
        seq = 'ACGTTGCATGTCGCATGATGCATGAGAGCT'
        out = ['GATG', 'ATGC', 'ATGT']
        res = find_frequent_kmers_in(seq, 4, 1)
        self.assertEqual(set(res), set(out))
    def test_2(self):
        seq = 'CACAGTAGGCGCCGGCACACACAGCCCCGGGCCCCGGGCCGCCCCGGGCCGGCGGCCGCCGGCGCCGGCACACCGGCACAGCCGTACCGGCACAGTAGTACCGGCCGGCCGGCACACCGGCACACCGGGTACACACCGGGGCGCACACACAGGCGGGCGCCGGGCCCCGGGCCGTACCGGGCCGCCGGCGGCCCACAGGCGCCGGCACAGTACCGGCACACACAGTAGCCCACACACAGGCGGGCGGTAGCCGGCGCACACACACACAGTAGGCGCACAGCCGCCCACACACACCGGCCGGCCGGCACAGGCGGGCGGGCGCACACACACCGGCACAGTAGTAGGCGGCCGGCGCACAGCC'
        out = ['GCACACAGAC', 'GCGCACACAC']
        res = find_frequent_kmers_in(seq, 10, 2)
        self.assertEqual(set(res), set(out))
    def test_appears(self):
        seq = 'AAAAAAAAAA'
        out = ['AA', 'AC', 'AG', 'CA', 'AT', 'GA', 'TA']
        res = find_frequent_kmers_in(seq, 2, 1)
        self.assertEqual(set(res), set(out))
    def test_swapping(self):
        seq = 'AGTCAGTC'
        out = ['TCTC', 'CGGC', 'AAGC', 'TGTG', 'GGCC', 'AGGT', 'ATCC', 'ACTG', 'ACAC', 'AGAG', 'ATTA', 'TGAC', 'AATT',
'CGTT', 'GTTC', 'GGTA', 'AGCA', 'CATC']
        res = find_frequent_kmers_in(seq, 4, 2)
        self.assertEqual(set(res), set(out))
    def test_complement(self):
        seq = 'AATTAATTGGTAGGTAGGTA'
        out = ['GGTA']
        res = find_frequent_kmers_in(seq, 4, 0)
        self.assertEqual(set(res), set(out))
    def test_cardinality(self):
        seq = 'ATA'
        out = ['GTA', 'ACA', 'AAA', 'ATC', 'ATA', 'AGA', 'ATT', 'CTA', 'TTA', 'ATG']
        res = find_frequent_kmers_in(seq, 3, 1)
        self.assertEqual(set(res), set(out))
    def test_complement_2(self):
        seq = 'AAT'
        out = ['AAT']
        res = find_frequent_kmers_in(seq, 3, 0)
        self.assertEqual(set(res), set(out))
    def test_last(self):
        seq = 'TAGCG'
        out = ['GG', 'TG']
        res = find_frequent_kmers_in(seq, 2, 1)
        self.assertEqual(set(res), set(out))

In [8]:
a = TestMismatch()

suite = unittest.TestLoader().loadTestsFromModule(a)
unittest.TextTestRunner().run(suite)

........
----------------------------------------------------------------------
Ran 8 tests in 0.493s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>